Это пример решения задачи. 
Можете скачать в формате ipynb
или сделать себе копию в Colab.

In [ ]:
!pip install hyppo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for hyppo: filename=hyppo-0.3.2-py3-none-any.whl size=134082 sha256=14d5fc64b313abdeedf406351476dc08ef9f922b1750ab37ebf4a5006393c6e7
  Stored in directory: /root/.cache/pip/wheels/da/7d/db/9b769ba455615350f1ce016c61e98388f6dc18e1be2403f826
Successfully built hyppo


In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import warnings

from hyppo.ksample import Energy, MMD, DISCO
from scipy.stats import laplace, norm, ks_2samp, anderson_ksamp, cramervonmises_2samp
from statsmodels.stats.weightstats import ztest
from statsmodels.distributions.empirical_distribution import ECDF


warnings.filterwarnings("ignore")

# Загрузите в Colab файлы из бота
historical_data = pd.read_csv("historical_data.csv") 
modified_data_of_type_1 = pd.read_csv("modified_data_of_type_1.csv")
modified_data_of_type_2 = pd.read_csv("modified_data_of_type_2.csv")
modified_data_of_type_3 = pd.read_csv("modified_data_of_type_3.csv")

data_list = [{
    "x": historical_data[::2],
    "y": historical_data[1::2],
    "dist_desc": "Историческое VS Историческое"
}, {
    "x": historical_data,
    "y": modified_data_of_type_1,
    "dist_desc": "Историческое VS Изменённое типа 1"
}, {
    "x": historical_data,
    "y": modified_data_of_type_2,
    "dist_desc": "Историческое VS Изменённое типа 2"
}, {
    "x": historical_data,
    "y": modified_data_of_type_3,
    "dist_desc": "Историческое VS Изменённое типа 3"
}]

test_list = [{
    "p_value": lambda x, y: ks_2samp(x, y, alternative="two-sided").pvalue,
    "name": "Kolmogorov-Smirnov"
}, {
    "p_value": lambda x, y: anderson_ksamp([x, y]).pvalue,
    "name": "Anderson"
}, {
    "p_value": lambda x, y: cramervonmises_2samp(x, y).pvalue,
    "name": "CVM"
}, {
    "p_value": lambda x, y: Energy(compute_distance="euclidean").test(x, y)[1],
    "name": "Energy with euclidean metric"
}, {
    "p_value": lambda x, y: MMD(compute_kernel="rbf", gamma=1/10).test(x, y)[1],
    "name": "MMD, rbf, gamma=0.1"
}, {
    "p_value": lambda x, y: MMD(compute_kernel="rbf", gamma=1).test(x, y)[1],
    "name": "MMD, rbf, gamma=1"
}, {
    "p_value": lambda x, y: MMD(compute_kernel="rbf", gamma=10).test(x, y)[1],
    "name": "MMD, rbf, gamma=10"
}]

for data in data_list:
    test_p_value_list = {
        test["name"]: []
        for test in test_list
    }

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=np.linspace(start=0, stop=1, num=1000), 
                             y=np.linspace(start=0, stop=1, num=1000),
                             mode="lines", name="y = x"))

    for i in range(data["x"].shape[0]):
        x = data["x"].iloc[i].to_numpy()
        y = data["y"].iloc[i].to_numpy()
        
        for test in test_list:
            test_p_value_list[test["name"]].append(test["p_value"](x, y))

    for test in test_list:
        p_value_ecdf = ECDF(test_p_value_list[test["name"]])
        fig.add_trace(go.Scatter(x=p_value_ecdf.x, y=p_value_ecdf.y,
                                 mode="lines", name=test["name"]))

    fig.update_layout(title=f"Сравнение различных методов для проверки однородности, {data['dist_desc']}",
                      xaxis_title="alpha",
                      yaxis_title="ЭФР p-value",
                      legend_title="Критерии")
    fig.show()

Смотрим что подходит под критерии из условия,
используя уровень значимости из задачи.
* Для "Историческое VS Историческое" значение графика в точке $\alpha$,
равной уровню значимости, есть вероятность ошибки первого рода.
* Для "Историческое VS Изменённое" значение графика в точке $\alpha$, 
равной уровню значимости, есть мощность
($1 - $ вероятность ошибки второго рода).